# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: Gil Garcera, Javier
*   Alumno 2: Palomares Mateo, Abel
*   Alumno 3: Serrano López, Francisco Rubén
*   Alumno 4: Vegas Romero, David







---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico  exists.

Colab: Changing directory to  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
Archivos en el directorio: 
['dqn_pong_50000_weights.h5f.data-00000-of-00001', 'dqn_pong_50000_weights.h5f.index', 'dqn_pong_100000_weights.h5f.data-00000-of-00001', 'dqn_pong_100000_weights.h5f.index', 'dqn_{}_weights.h5f.index', 'dqn_{}_weights.h5f.data-00000-of-00001', 'checkpoint']


---
### 1.4. Instalar librerías necesarias

In [3]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12.1
else:
  %pip install numpy==1.23.5
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.12.0
  %pip install tensorflow==2.12.1
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-mdvzzcp3
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-mdvzzcp3
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [4]:
from __future__ import division

from PIL import Image
import numpy as np
import gym


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

# Callback personalizado para guardar pesos cada 50.000 pasos
from keras.callbacks import Callback
import os

#### Configuración base

In [5]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [6]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [18]:
def build_model(nb_actions, input_shape, window_length):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))  # (4, 84, 84) → (84, 84, 4)

    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))  # output Q-values para cada acción

    print(model.summary())
    return model

2. Implementación de la solución DQN

In [19]:
model = build_model(nb_actions, INPUT_SHAPE, WINDOW_LENGTH)

# Memoria y política
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(), attr='eps',
    value_max=1.0, value_min=0.05, value_test=0.01,
    nb_steps=500000
)

dqn = DQNAgent(
    model=model,
    nb_actions=nb_actions,
    policy=policy,
    memory=memory,
    processor=AtariProcessor(),
    nb_steps_warmup=50000,
    gamma=0.99,
    target_model_update=10000,
    train_interval=4,
    delta_clip=1.0,
    enable_double_dqn=True,
    enable_dueling_network=True,
    dueling_type='avg',
    batch_size=16
)


dqn.compile(Adam(learning_rate=0.00025), metrics=['mae'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute_4 (Permute)         (None, 84, 84, 4)         0         
                                                                 
 conv2d_9 (Conv2D)           (None, 20, 20, 32)        8224      
                                                                 
 conv2d_10 (Conv2D)          (None, 9, 9, 64)          32832     
                                                                 
 conv2d_11 (Conv2D)          (None, 7, 7, 64)          36928     
                                                                 
 flatten_4 (Flatten)         (None, 3136)              0         
                                                                 
 dense_12 (Dense)            (None, 512)               1606144   
                                                                 
 dense_13 (Dense)            (None, 6)                

**CALLBACK PARA GUARDAR PESOS + ENTRENAMIENTO**

In [12]:
class SaveWeightsCallback(Callback):
    def __init__(self, dqn, prefix='dqn_weights', interval=50000):
        self.dqn = dqn
        self.prefix = prefix
        self.interval = interval
        self.steps = 0

    def on_step_end(self, step, logs={}):
        self.steps += 1
        if self.steps % self.interval == 0:
            filename = f'{self.prefix}_{self.steps}_weights.h5f'
            print(f"\nGuardando pesos en {filename}")
            self.dqn.save_weights(filename, overwrite=True)

save_callback = SaveWeightsCallback(dqn, prefix='dqn_improved', interval=50000)


**ENTRENAMIENTO**

In [ ]:
dqn.fit(
    env,
    nb_steps=1750000,
    log_interval=10000,
    callbacks=[save_callback]
)

Training for 1750000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 0.0157
13 episodes - episode_reward: 11.769 [5.000, 26.000] - ale.lives: 1.940

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 0.0138
14 episodes - episode_reward: 9.786 [5.000, 22.000] - ale.lives: 2.119

Interval 3 (20000 steps performed)
 7739/10000 [======================>.......] - ETA: 9s - reward: 0.0127
Guardando pesos en dqn_improved_50000_weights.h5f
10000/10000 [==============================] - 44s 4ms/step - reward: 0.0133
15 episodes - episode_reward: 8.533 [4.000, 15.000] - ale.lives: 1.991

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 0.0125
14 episodes - episode_reward: 9.571 [1.000, 17.000] - ale.lives: 1.975

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 0.0148
14

In [ ]:
# Cargar los mejores pesos (opcional si no se quiere continuar desde el último entrenado)
# dqn.load_weights('dqn_improved_1500000_weights.h5f')

# Evaluar durante 20 episodios
history = dqn.test(
    env,
    nb_episodes=20,
    visualize=True,
    verbose=2
)

In [9]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for dqn_SpaceInvaders-v0_weights.h5f

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---